In [2]:
import json
import xmltodict
from copy import *

In [3]:
#Recuperation du manifest

manifest = (open("manifest.txt")).read()
lines = manifest.split('\n')
keys = lines[0].split('\t')
rows = []
for i in range (1,len(lines)) :
    row = lines[i].split('\t')
    rows.append(row)

columns = []

for j in range (len(rows[0])) : 
    column = []
    for i in range (len(rows)) :
        column.append(rows[i][j])
    columns.append(column)
    
list_dict = []

for j in range (len(keys)) :
    list_dict.append((keys[j],columns[j]))
    
manifest = dict(list_dict)


In [7]:
print(manifest.keys())
len(manifest['filename'])

dict_keys(['id', 'filename', 'md5', 'size', 'state'])


9611

In [9]:
#Ecriture des fichiers json

n = len(manifest['id']) #nombre de fichiers

for i in range (n) : 
    
    dossier = manifest['id'][i]
    fichier = manifest['filename'][i]
    
    with open("data/"+dossier+"/"+fichier) as xml_file :
        data_dict = xmltodict.parse(xml_file.read())
        xml_file.close()
        json_data = json.dumps(data_dict)

        with open("data/"+dossier+"/"+fichier[:-3]+"json","w") as json_file :
            json_file.write(json_data)
            json_file.close()
    

In [11]:
#Transformation des fichiers

for i in range (n) : 
    
    dossier = manifest['id'][i]
    fichier = manifest['filename'][i][:-3]+"json"

    #charge le fichier json
    file = json.load(open("data/"+dossier+"/"+fichier))

    #recupere uniquement la cle 'tcga_bcr'
    keys = file.keys()
    for key in keys : 
        if 'tcga_bcr' in key :
            old_key = key
    file=file[old_key]

    #recupere uniquement la cle 'patient'
    keys = file.keys()
    for key in keys : 
        if 'patient' in key :
            old_key = key
    file=file[old_key]

    #simplifie les cles
    keys = file.keys()
    filebis = {}

    for key in keys :
        key_decomposition = key.split(':')

        if (len(key_decomposition) == 2) :
            new_key = key_decomposition[1]
            filebis[new_key] = file[key]
            if type(filebis[new_key]) == dict :
                if '@procurement_status' in filebis[new_key].keys() :
                    if filebis[new_key]['@procurement_status'] != 'Completed' :
                        filebis[new_key] = None
                    elif '#text' in filebis[new_key].keys() : 
                        filebis[new_key] = filebis[new_key]['#text']

    #ecrit le nouveau fichier simplifié
    with open("data/"+dossier+"/new_"+fichier,"w") as json_file :
        new = json.dumps(filebis)
        json_file.write(new)
        json_file.close()


In [12]:
#Construction d'un unique fichier avec tous les patients

final_file = []

for i in range (n) : 
    
    dossier = manifest['id'][i]
    fichier = "new_"+manifest['filename'][i][:-3]+"json"
    
    file = json.load(open("data/"+dossier+"/"+fichier))
    final_file.append(file)
    
with open("data.json","w") as json_file :
    data = json.dumps(final_file)
    json_file.write(data)
    json_file.close()
    

Steps facultatifs pour consulter le fichier final

In [13]:
#Tests sur les infos du fichier final

data = json.load(open('data.json'))


In [18]:
#Cherche les clés communes à tous les fichiers

common_keys_i = data[0].keys()
for i in range (1,n):
    keys_i = data[i].keys()
    common_keys_ip1 = []
    for key in common_keys_i :
        if key in keys_i :
            common_keys_ip1.append(key)
    common_keys_i = common_keys_ip1
    
common_keys_i
    

['additional_studies',
 'other_dx',
 'gender',
 'vital_status',
 'days_to_birth',
 'days_to_death',
 'days_to_last_followup',
 'race_list',
 'bcr_patient_barcode',
 'tissue_source_site',
 'patient_id',
 'bcr_patient_uuid',
 'history_of_neoadjuvant_treatment',
 'informed_consent_verified',
 'icd_o_3_site',
 'icd_o_3_histology',
 'icd_10',
 'tissue_prospective_collection_indicator',
 'tissue_retrospective_collection_indicator',
 'ethnicity',
 'person_neoplasm_cancer_status',
 'day_of_form_completion',
 'month_of_form_completion',
 'year_of_form_completion',
 'new_tumor_events',
 'drugs',
 'radiations',
 'follow_ups']